In [3]:
include("CGS_POMDP.jl");
pomdp = CGSPOMDP();

### Abstract GPs Scratch code

In [14]:
function observe(pomdp::CGSPOMDP, point::Point, layer::Int, column::Symbol)
    f = pomdp.belief[layer][column]
    x = pcu(point)
    y = pomdp.state.earth[layer].gt[point, column]
    p_fx = posterior(f(x, 0.1), y)
    pomdp.belief[layer][column] = p_fx

    return p_fx(x, 0.1)
end

Base.Meta.ParseError: ParseError:
# Error @ /Users/suhaschundi/Documents/CGSSISL/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:8:24

    return p_fx(x, 0.1)
#                      └ ── Expected `end`

In [2]:
function visualize_uncertainty(pomdp::CGSPOMDP)
    df = copy(pomdp.state.earth[1].df)
    dom = domain(pomdp.state.earth[1].gt)

    for layer in 1:NUM_LAYERS
        gtlayer = pomdp.state.earth[layer].gt
        data_at_all_wells = gtlayer[Multi([pomdp.collected_locs...]), :]

        γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET) # Each feature can have a different nugget in the future.
        okrig = GeoStatsModels.OrdinaryKriging(γ)
        fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

        for column in pomdp.feature_names
            var_column = Symbol("$(column)_$(layer)_variances")
            df[!, var_column] = [GeoStatsModels.predictprob(fitkrig, column, pt).σ
                            for pt in domain(pomdp.state.earth[1].gt)]
        end
    end
    var_map = georef(df, dom)
    viewer(var_map)
end

visualize_uncertainty (generic function with 1 method)

### Test: Action function returns all actions

In [3]:
all_actions = POMDPs.actions(pomdp);

### Test: Observation function returns correct distributions

In [4]:
POMDPs.observation(pomdp, all_actions[2], pomdp.state)

DiagNormal(
dim: 15
μ: [-3.3839821020512524, 76.4781855498032, 75.81037484397422, 5.909107004440592, 117.81757339165631, 67.12876568798308, 13.0959934099886, 77.6823174426963, 26.75527952482024, 23.041993556294873, 161.54534436973438, 78.72169590809182, 30.385034972276202, 58.00357549460046, 74.7706573393428]
Σ: [0.6617649993262633 0.0 … 0.0 0.0; 0.0 0.6617649993262633 … 0.0 0.0; … ; 0.0 0.0 … 0.6617649993262633 0.0; 0.0 0.0 … 0.0 0.6617649993262633]
)


In [5]:
POMDPs.reward(pomdp, pomdp.state, all_actions[2])

9.0

In [6]:
visualize_uncertainty(pomdp)

In [7]:
POMDPs.observation(pomdp, all_actions[21], pomdp.state) # Can take up to 7 mins to run!

Distributions.ProductDistribution{2, 1, Vector{MvNormal}, Distributions.Continuous, Float64}(
dists: MvNormal[FullNormal(
dim: 20
μ: [1.4566647061421922, 3.403922569850415, 3.6617521036927196, 2.2619706584048456, 1.5313036600066148, -0.16699835395039025, -0.06029857614342938, 1.2886710724856782, 0.7727233174025183, -0.26533002400307415, -0.7060660403285841, 0.12241905925019678, 0.7366427215102422, 0.9299712308720696, 2.851685551412363, 3.3130112280333104, 3.8884975076738404, 2.7420089958215494, 3.205076829045678, 2.978364315331161]
Σ: [1.0e-5 2.0055785725957926e-30 … -2.8078100016341097e-30 -2.406694287114951e-30; 2.0055785725957926e-30 1.0e-5 … -7.019525004085275e-30 -6.016735717787378e-30; … ; -2.8078100016341097e-30 -7.019525004085275e-30 … 1.0e-5 8.42343000490233e-30; -2.406694287114951e-30 -6.016735717787378e-30 … 8.42343000490233e-30 1.0e-5]
)
, FullNormal(
dim: 20
μ: [75.37223448730896, 77.96678888526225, 76.41210281021264, 77.64483765090851, 78.23520280693303, 79.88732526451592

In [8]:
POMDPs.reward(pomdp, pomdp.state, all_actions[21])

395.0

In [9]:
visualize_uncertainty(pomdp)

### Test: Points close are predicted with more accuracy and less uncertainty than points far.

In [8]:
gtlayer = pomdp.state.earth[1].gt

# seismic_line = (x1=0.0, y1=0.0, x2=5.0, y2=98.0)
well1 = Point(19, 34)
well2 = Point(30, 12)
well_close = Point(31, 14)
well_far = Point(98, 97)

all_wells = Multi([well1, well2])
data_at_all_wells = gtlayer[all_wells, :]

γ = SphericalVariogram(range=RANGE, sill=SILL, nugget=NUGGET)

okrig = GeoStatsModels.OrdinaryKriging(γ)
fitkrig = GeoStatsModels.fit(okrig, data_at_all_wells)

# We notice that prediction at close has much less variance than prediction at far, 
# and as an additional sanity check the prediction at close is close to the well2 point.
# Krigging predictprob only does one attribute at a time.

probs_far = GeoStatsModels.predictprob(fitkrig, :z, well_far)
probs_close = GeoStatsModels.predictprob(fitkrig, :z, well_close)
probs_close, probs_far

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Test: Uncertainty is low around known points.
Additionally, as nugget is increased uncertainty (both globally and at known points) increases.
Pay attention to colorbar when verifying this.

### Buy well data at 3-4 locations and visualize belief

In [10]:
buy_well_data(pomdp, 2)
buy_well_data(pomdp, 5)
buy_well_data(pomdp, 7)

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
visualize_uncertainty(Multi([pomdp.collected_locs...]), 3, "z") # Note this is time consuming at just 3 gathered points.

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
distr = observe(pomdp, Point(18, 24), 3, "z")
rand(distr)

UndefVarError: UndefVarError: `pomdp` not defined in `Main`
Suggestion: check for spelling errors or missing imports.